In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Tina Turner,Look Me In The Heart,Last night we tried to touch but we never got ...
1,Richard Marx,What's Wrong With That,I may be rotten to the core \r\nAnd stop me i...
2,Santana,Brown Skin Girl,She's a force of nature that I can't outrun \...
3,Spandau Ballet,Round And Round,I don't have to be so wise \r\nYou're just my...
4,The White Stripes,Screwdriver,Tuesday mornin' now \r\nI gotta have somewher...
5,Olivia Newton-John,The Way You Look Tonight,Someday when I'm awfully low \r\nWhen the wor...
6,Phish,Friday,I crashed a \r\nLearned to keep my eye on you...
7,Quarashi,Baseline,I give a shit about the kick that's been comin...
8,Dave Matthews Band,Blue Water Baboon Farm,Walking by the river's edge \r\nReach my hand...
9,Cheap Trick,No Mercy,No mercy \r\nNo mercy \r\nNo mercy \r\n \r...


In [12]:
df['text'][0]

"last night we tried to touch but we never got close  \r last night we tried to talk, the words got caught in our throats  \r when we finally fell asleep  \r we couldn't have been further apart  \r   \r look me in the heart  \r if you think that love is blind  \r baby look me in the heart  \r and you'll see that i'm so crazy about you baby  \r and it's not in my mind  \r can't you look me in the heart  \r look me in the heart  \r   \r you try to say that i'm hiding from you  \r you act like a spy always looking for clues  \r you've read about my past  \r but why don't we try to make a new start  \r   \r look me in the heart  \r if you think that love is blind  \r baby look me in the heart  \r and you'll see that i'm so crazy about you baby  \r and it's not in my mind  \r can't you look me in the heart  \r look me in the heart  \r   \r remember how good we used to be together  \r remember baby, so be tender baby  \r remember the love we said would last forever  \r i know we can make it 

In [10]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [11]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.03953396, 0.06036128, ..., 0.01641652, 0.01573023,
       0.01856211])

In [18]:
df[df['song'] == 'Crying Over You']

,artist,song,text
3304,ABBA,Crying Over You,i 'm waitin ' for you babi i 'm sit all alon i...


In [22]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:5]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [23]:
recommendation('Crying Over You')

["Cryin'",
 'The Crying Game',
 "Cryin' Time",
 "Don't Let The Sun Catch You Crying"]